### **If you are using Colab for free, we highly recommend you active the T4 GPU hardware accelerator. Our models are designed to run with at least 16GB of RAM, activating T4 will grant the notebook 16GB of GDDR6 RAM as opposed to the 13GB Colab gives automatically.**
### **To active T4:**
### **1. click on the "Runtime" tab**
### **2. click on "Change runtime type"**
### **3. select T4 GPU under Hardware Accelerator**
### **NOTE: there is a weekly usage limit on using T4 for free**

# **Fast Start Example 9 - Function Calls with Web Services** #
This example shows how function calls can be combined with external web services for pattern auotomation. Three function calling models will work together with two web services to output a structured analysis of a company. Locally-run LLM calls will be utilized to extract information out of Wikipedia articles.

Models:
1.   slim-extract-tool
2.   slim-summary-tool
3.   bling-stablelm-3b-tool

Web Services
1.  Yfinance - stock ticker
2.  Wikipedia - company background information



In [6]:
!pip install llmware

In [7]:
from llmware.web_services import YFinance
from llmware.models import ModelCatalog
from llmware.parsers import WikiParser

from importlib import util
if not util.find_spec("yfinance"):
    print("\nto run this example, you need to install yfinance first, e.g., pip3 install yfinance")

In [8]:
!pip3 install yfinance

Sample text from CNBC earnings release

In [9]:
text=("BEAVERTON, Ore.--(BUSINESS WIRE)--NIKE, Inc. (NYSE:NKE) today reported fiscal 2024 financial results for its "
      "third quarter ended February 29, 2024.) “We are making the necessary adjustments to drive NIKE’s next chapter "
      "of growth Post this Third quarter revenues were slightly up on both a reported and currency-neutral basis* "
      "at $12.4 billion NIKE Direct revenues were $5.4 billion, slightly up on a reported and currency-neutral basis "
      "NIKE Brand Digital sales decreased 3 percent on a reported basis and 4 percent on a currency-neutral basis "
      "Wholesale revenues were $6.6 billion, up 3 percent on a reported and currency-neutral basis Gross margin "
      "increased 150 basis points to 44.8 percent, including a detriment of 50 basis points due to restructuring charges "
      "Selling and administrative expense increased 7 percent to $4.2 billion, including $340 million of restructuring "
      "charges Diluted earnings per share was $0.77, including $0.21 of restructuring charges. Excluding these "
      "charges, Diluted earnings per share would have been $0.98* “We are making the necessary adjustments to "
      "drive NIKE’s next chapter of growth,” said John Donahoe, President & CEO, NIKE, Inc. “We’re encouraged by "
      "the progress we’ve seen, as we build a multiyear cycle of new innovation, sharpen our brand storytelling and "
      "work with our wholesale partners to elevate and grow the marketplace.")

Load three models

In [10]:
model = ModelCatalog().load_model("slim-extract-tool", temperature=0.0, sample=False)
model2 = ModelCatalog().load_model("slim-summary-tool", sample=False,temperature=0.0,max_output=200)
model3 = ModelCatalog().load_model("bling-stablelm-3b-tool", sample=False, temperature=0.0)

research_summary = {}

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1194: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.96k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/38.8k [00:00<?, ?B/s]

...

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/247k [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

bling-stablelm.gguf:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

#slim-extract-tool#


Extract important information

In [11]:
extract_keys = ["stock ticker", "company name",
                "total revenues", "restructuring charges",
                "digital growth", "ceo comment", "quarter end date"]
print("\nStep 1 - extract information from source text\n")


Step 1 - extract information from source text



In [12]:
for keys in extract_keys:
    response = model.function_call(text,params=[keys])
    dict_keys = keys.replace(" ", "_")
    print(f"update: extracting - {keys} - {response['llm_response']}")
    if dict_keys in response["llm_response"]:
        value = response["llm_response"][dict_keys][0]
        research_summary.update({dict_keys: value})
    else:
        print("could not find look up key successfully - ", response["llm_response"])

update: extracting - stock ticker - {'stock_ticker': ['NYSE:NKE']}
update: extracting - company name - {'company_name': ['NIKE, Inc.']}
update: extracting - total revenues - {'total_revenues': ['$12.4 billion']}
update: extracting - restructuring charges - {'restructuring_charges': ['$340 million']}
update: extracting - digital growth - {'digital_growth': ['NIKE Brand Digital sales decreased 3% on a reported basis and 4% on a currency-neutral basis.']}


INFO:llmware.models:update: function call output could not be automatically converted, but remediation was successful to type - dict 


update: extracting - ceo comment - {'ceo_comment': ['NIKE is making the necessary adjustments to drive NIKEs next chapter of growth.']}
update: extracting - quarter end date - {'quarter_end_date': ['February 29, 2024']}


#Secondary lookups and summary#





In [13]:
print("\nStep 2 - use extracted stock ticker in web service lookup to YFinance\n")


Step 2 - use extracted stock ticker in web service lookup to YFinance



Add to research summary dictionary

In [14]:
if "stock_ticker" in research_summary:
        ticker = research_summary["stock_ticker"]
        # a little kludge related to yfinance api
        ticker_core = ticker.split(":")[-1]

        yf = YFinance().get_stock_summary(ticker=ticker_core)
        print("yahoo finance stock info: ", yf)

        research_summary.update({"current_stock_price": yf["currentPrice"]})
        research_summary.update({"high_ltm": yf["fiftyTwoWeekHigh"]})
        research_summary.update({"low_ltm": yf["fiftyTwoWeekLow"]})
        research_summary.update({"trailing_pe": yf["trailingPE"]})
        research_summary.update({"forward_pe": yf["forwardPE"]})
        research_summary.update({"volume": yf["volume"]})

        yf2 = YFinance().get_financial_summary(ticker=ticker_core)
        print("yahoo finance fin info - ", yf2)
        research_summary.update({"market_cap": yf2["marketCap"]})
        research_summary.update({"price_to_sales": yf2["priceToSalesTrailing12Months"]})
        research_summary.update({"revenue_growth": yf2["revenueGrowth"]})
        research_summary.update({"ebitda": yf2["ebitda"]})
        research_summary.update({"gross_margin": yf2["grossMargins"]})
        research_summary.update({"currency": yf2["currency"]})

        yf3 = YFinance().get_company_summary(ticker=ticker_core)
        print("yahoo finance company info - ", yf3)
        research_summary.update({"sector": yf3["sector"]})
        research_summary.update({"website": yf3["website"]})
        research_summary.update({"industry": yf3["industry"]})
        research_summary.update({"employees": yf3["fullTimeEmployees"]})

        execs = []
        if "companyOfficers" in yf3:
            for entries in yf3["companyOfficers"]:
                if "totalPay" in entries:
                    pay = entries["totalPay"]
                else:
                    pay = "pay-NA"

                if "age" in entries:
                    age = entries["age"]
                else:
                    age = "age-NA"

                execs.append((entries["name"], entries["title"], age, pay))
        research_summary.update({"officers": execs})

yahoo finance stock info:  {'shortName': 'Nike, Inc.', 'symbol': 'NKE', 'exchange': 'NYQ', 'bid': 73.8, 'ask': 73.98, 'fiftyTwoWeekLow': 70.91, 'fiftyTwoWeekHigh': 123.39, 'longName': 'NIKE, Inc.', 'currentPrice': 74.01, 'targetHighPrice': 120.0, 'targetLowPrice': 60.0, 'returnOnAssets': 0.11161, 'returnOnEquity': 0.40093, 'trailingPE': 19.841824, 'forwardPE': 20.6156, 'volume': 10023093, 'forwardEps': 3.59, 'pegRatio': 4.81, 'currency': 'USD'}
yahoo finance fin info -  {'shortName': 'Nike, Inc.', 'symbol': 'NKE', 'marketCap': 110967635968, 'totalRevenue': 51362000896, 'ebitda': 7598000128, 'revenueGrowth': -0.017, 'grossMargins': 0.44685, 'freeCashflow': 5901125120, 'priceToSalesTrailing12Months': 2.1605005, 'currency': 'USD'}
yahoo finance company info -  {'shortName': 'Nike, Inc.', 'longName': 'NIKE, Inc.', 'symbol': 'NKE', 'marketCap': 110967635968, 'companyOfficers': [{'maxAge': 1, 'name': 'Mr. Mark G. Parker', 'age': 67, 'title': 'Executive Chairman', 'yearBorn': 1956, 'fiscalYea

#Search name -> info -> ask questions#

In [15]:
print("\nStep 3 - use extracted company name to lookup in Wikipedia web service - and add background data\n")


Step 3 - use extracted company name to lookup in Wikipedia web service - and add background data



In [16]:
!pip install wikipedia-api

In [17]:
if "company_name" in research_summary:

    company_name = research_summary["company_name"]
    output = WikiParser().add_wiki_topic(company_name, target_results=1)

    #   get company summary
    company_overview = ""
    for i, blocks in enumerate(output["blocks"]):
        if i < 3:
          company_overview += blocks["text"]

    # call summary model to summarize
    print("-- calling summary model to summarize the first part of the Wikipedia article")
    summary = model2.function_call(company_overview, params=["company history (5)"])
    print("-- slim-summary - summary (5 points): ", summary)

    research_summary.update({"summary": summary["llm_response"]})

    #  get founding date
    print("\n-- calling extract model to get key piece of information from the Wikipedia article - company founding date")
    response = model.function_call(company_overview, params=["founding date"])
    print("-- slim-extract - founding date: ", response)

    research_summary.update({"founding_date": response["llm_response"]["founding_date"][0]})

    print("\n-- calling extract model to get a short company business")
    response = model.function_call(company_overview, params=["company description"])
    print("-- slim-extract - response: ", response)
    research_summary.update({"company_description": response["llm_response"]["company_description"][0]})

    #     ask other questions directly
    print("\n-- asking a question directly to the Wikipedia article about the business")
    response = model3.inference("What is an overview of company's business?", add_context=company_overview)
    print("-- bling-answer model - response: ", response)
    research_summary.update({"business_overview": response["llm_response"] })

    print("\n-- asking a question about the origin of the company's name")
    response = model3.inference("What is the origin of the company's name?", add_context=company_overview)
    print("-- bling-answer model - response: ", response)
    research_summary.update({"origin_of_name": response["llm_response"]})

    print("\n-- asking a question about the company's products")
    response = model3.inference("What are the product names", add_context=company_overview)
    print("-- bling-answer model - response: ", response)
    research_summary.update({"products": response["llm_response"]})

    print("\n\nStep 4 - Completed Research - Summary Output\n")
    print("research summary: ", research_summary)

    item_counter = 1

    for keys, values in research_summary.items():
        if isinstance(values, str):

            values = values.replace("\n", "")
            values = values.replace("\r", "")
            values = values.replace("\t", "")

        print(f"\t\t -- {item_counter} - \t - {keys.ljust(25)} - {str(values).ljust(40)}")
        item_counter += 1

-- calling summary model to summarize the first part of the Wikipedia article


INFO:llmware.models:update: function call output could not be automatically converted, but remediation was successful to type - list 


-- slim-summary - summary (5 points):  {'llm_response': ['Nike Inc. was founded on January 25 1964 as ', 'by Bill Bowerman and Phil Knight', 'The company was officially named Nike Inc. on May 30 1971', 'Nike is the worlds largest supplier of athletic shoes and apparel and a major manufacturer of sports equipment', 'Nike sponsors many high-profile athletes and sports teams around the world', 'Nike is the most valuable brand among sports businesses'], 'usage': {'input': 380, 'output': 97, 'total': 477, 'metric': 'tokens', 'processing_time': 8.21936297416687, 'type': 'list', 'remediation': True}, 'logits': [[(13961, 1.0), (187, 0.0), (18256, 0.0), (1966, 0.0), (8801, 0.0), (78, 0.0), (14, 0.0), (67, 0.0), (15, 0.0), (1342, 0.0)], [(47, 0.661), (39, 0.171), (25310, 0.045), (8252, 0.02), (510, 0.019), (13652, 0.013), (2374, 0.012), (22423, 0.009), (32088, 0.006), (3, 0.004)], [(2804, 0.999), (6541, 0.0), (13972, 0.0), (587, 0.0), (366, 0.0), (460, 0.0), (1758, 0.0), (8678, 0.0), (6840, 0.0)

INFO:llmware.models:update: function call output could not be automatically converted, but remediation was successful to type - dict 


-- slim-extract - response:  {'llm_response': {'company_description': ['Nike Inc. is the worlds largest supplier of athletic shoes and apparel and a major manufacturer of sports equipment.']}, 'usage': {'input': 373, 'output': 33, 'total': 406, 'metric': 'tokens', 'processing_time': 3.67295241355896, 'type': 'dict', 'remediation': True}, 'logits': [[(13961, 0.999), (10816, 0.0), (73, 0.0), (1087, 0.0), (12080, 0.0), (18256, 0.0), (10984, 0.0), (8656, 0.0), (14077, 0.0), (1342, 0.0)], [(25610, 1.0), (25590, 0.0), (32729, 0.0), (10008, 0.0), (10816, 0.0), (1590, 0.0), (14267, 0.0), (38885, 0.0), (46723, 0.0), (4674, 0.0)], [(64, 1.0), (876, 0.0), (5295, 0.0), (14, 0.0), (4414, 0.0), (3333, 0.0), (11185, 0.0), (15, 0.0), (27, 0.0), (47499, 0.0)], [(10008, 1.0), (3229, 0.0), (1590, 0.0), (12898, 0.0), (39930, 0.0), (28692, 0.0), (49027, 0.0), (20119, 0.0), (39990, 0.0), (14729, 0.0)], [(5295, 0.999), (47499, 0.001), (64, 0.0), (27, 0.0), (1381, 0.0), (6038, 0.0), (8, 0.0), (876, 0.0), (545

In [19]:
print(research_summary)

{'stock_ticker': 'NYSE:NKE', 'company_name': 'NIKE, Inc.', 'total_revenues': '$12.4 billion', 'restructuring_charges': '$340 million', 'digital_growth': 'NIKE Brand Digital sales decreased 3% on a reported basis and 4% on a currency-neutral basis.', 'ceo_comment': 'NIKE is making the necessary adjustments to drive NIKEs next chapter of growth.', 'quarter_end_date': 'February 29, 2024', 'current_stock_price': 74.01, 'high_ltm': 123.39, 'low_ltm': 70.91, 'trailing_pe': 19.841824, 'forward_pe': 20.6156, 'volume': 10023093, 'market_cap': 110967635968, 'price_to_sales': 2.1605005, 'revenue_growth': -0.017, 'ebitda': 7598000128, 'gross_margin': 0.44685, 'currency': 'USD', 'sector': 'Consumer Cyclical', 'website': 'https://www.nike.com', 'industry': 'Footwear & Accessories', 'employees': 79400, 'officers': [('Mr. Mark G. Parker', 'Executive Chairman', 67, 6008438), ('Mr. John J. Donahoe II', 'President, CEO & Director', 62, 9946993), ('Mr. Matthew  Friend', 'Executive VP & CFO', 45, 2290408),